In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from utility import *
from models import *

In [2]:
import math

import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
import torch.utils.data as data
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor

import torch.optim as optim
import torch.nn.functional as F

In [3]:
from os import listdir
from os.path import isfile, join
import numpy as np

polycube_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/Generated-Cars/"
polycube_files = [f for f in listdir(polycube_path) if isfile(join(polycube_path, f))]

voxelized_mesh_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/voxelizedMeshes/"
voxelized_mesh_files = [f for f in listdir(voxelized_mesh_path) if isfile(join(voxelized_mesh_path, f))]

voxelizedFiles = []
polycubedFiles = []

for f in voxelized_mesh_files: 
    if f[-13:] == "voxelized.txt":
        voxelizedFiles = np.hstack((voxelizedFiles, f))
    
for f in polycube_files:
    if f[-14:] == "finalCubes.txt":
        polycubedFiles = np.hstack((polycubedFiles, f))

# Define the global parameters 

In [4]:
grid_size=32
batch_size=15

### Save the tensor to a text file 

In [5]:
voxelized_train_input, polycube_target=loadData(grid_size, polycube_path, voxelized_mesh_path, voxelizedFiles, polycubedFiles, loadFromScratch=True)

<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch

# Load Data

In [6]:
preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target = preprocessing_train(voxelized_train_input, polycube_target,batch_size, False, False)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train)
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation)
preprocessed_input_train_target = torch.from_numpy(preprocessed_input_train_target)
preprocessed_input_validation_target = torch.from_numpy(preprocessed_input_validation_target)

Ntrain = len(preprocessed_input_train[:, 0,0,0,0]) 
Nvalidation = len(preprocessed_input_validation[:,0,0,0,0])

train_input = Variable(preprocessed_input_train.view(Ntrain, 1, grid_size, grid_size, grid_size).float())
validation_input = Variable(preprocessed_input_validation.view(Nvalidation, 1, grid_size, grid_size, grid_size).float())

labels_train = preprocessed_input_train_target.float()
labels_validation = preprocessed_input_validation_target.float()

print('train', train_input.shape)
print('validation', validation_input.shape)
print('train_target', labels_train.shape)
print('validation_target', labels_validation.shape)

train torch.Size([45, 1, 32, 32, 32])
validation torch.Size([15, 1, 32, 32, 32])
train_target torch.Size([45, 32, 32, 32])
validation_target torch.Size([15, 32, 32, 32])


# Network initialization

In [ ]:
# Train network 
#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
#criterion = nn.PoissonNLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss() #interesting ... but does not converge
#criterion = nn.MSELoss() #0.83 but unstable

if isinstance(criterion, nn.CrossEntropyLoss):
    train_target = Variable(preprocessed_input_train_target).long()  # keep long tensors
    validation_target = Variable(preprocessed_input_validation_target, volatile=True).long() # convert to float
    Noutputs = 2
    
elif isinstance(criterion, nn.NLLLoss):
    train_target = Variable(preprocessed_input_train_target)  # keep long tensors
    validation_target = Variable(preprocessed_input_validation_target, volatile=True) # convert to float
    Noutputs = 2
    
else:
    train_target = Variable(preprocessed_input_train_target.float()) # convert to float
    validation_target = Variable(preprocessed_input_validation_target.float(), volatile=True ) # convert to float
    Noutputs = 1
    
Nbatches = int(math.ceil(Ntrain/batch_size)) #batch_size is defined above
Nepochs = 1000
Nrep = 1
        
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
model = UnetGenerator_3d(in_dim=1, out_dim=Noutputs, num_filter=4)

#optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.90)
#optimizer = optim.Adam(model.parameters())
#optimizer = optim.Adagrad(model.parameters())
#optimizer = optim.Adamax(model.parameters())
#optimizer = optim.ASGD(model.parameters())
optimizer = optim.RMSprop(model.parameters())
#optimizer = optim.Rprop(model.parameters())
 
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True) #Reduces the learning rate if it did not decreased by more than 10^-4 in 10 steps

train_errors = torch.Tensor(Nepochs).zero_()
validation_errors = torch.Tensor(Nepochs).zero_()

ep_loss = torch.Tensor(Nepochs).zero_()

for i_ep in range(Nepochs):
    for b_start in range(0, Ntrain, batch_size):
        bsize_eff = batch_size - max(0, b_start+batch_size-Ntrain)  # boundary case
        model.train()
        model.zero_grad()
        output = model(train_input.narrow(0, b_start, bsize_eff))
        if isinstance(criterion, nn.CrossEntropyLoss) or isinstance(criterion, nn.NLLLoss):
            batch_loss = criterion(output, train_target.narrow(0, b_start, bsize_eff))
        else:
            #if delta model is chosen
            #batch_loss = criterion(output.view(bsize_eff*Noutputs), train_target.narrow(0, b_start, bsize_eff))
            batch_loss = criterion(output.view(bsize_eff,grid_size,grid_size,grid_size), train_target.narrow(0, b_start, bsize_eff))
        ep_loss[i_ep] += batch_loss.data[0]
        batch_loss.backward()
        optimizer.step()

    scheduler.step(ep_loss[i_ep])

    nb_train_errs = compute_nb_errors(model, train_input, train_target, batch_size, criterion)
    nb_validation_errs = compute_nb_errors(model, validation_input, validation_target, batch_size, criterion)

    Ntrain_nb = Ntrain*grid_size**3
    Nvalidation_nb = Nvalidation*grid_size**3
    print("Epoch Number : ", i_ep)
    print("\t Training accuracy: ", (100*(Ntrain_nb-nb_train_errs)/Ntrain_nb))
    print("\t Validation accuracy ",(100*(Nvalidation_nb-nb_validation_errs)/Nvalidation_nb))

    print("\t Epoch Loss ", ep_loss[i_ep])

    train_errors[i_ep] = nb_train_errs
    validation_errors[i_ep] = nb_validation_errs


------Initiating U-Net------

Epoch Number :  0
	 Training accuracy:  80.7977294921875
	 Validation accuracy  80.31778971354167
	 Epoch Loss  1.9901957511901855
Epoch Number :  1
	 Training accuracy:  81.395263671875
	 Validation accuracy  80.91634114583333
	 Epoch Loss  1.9006342887878418
Epoch Number :  2
	 Training accuracy:  85.98971896701389
	 Validation accuracy  85.61055501302083
	 Epoch Loss  1.8481688499450684
Epoch Number :  3
	 Training accuracy:  83.41959635416667
	 Validation accuracy  83.12459309895833
	 Epoch Loss  1.8189024925231934
Epoch Number :  4
	 Training accuracy:  89.18613009982639
	 Validation accuracy  89.41752115885417
	 Epoch Loss  1.7748970985412598
Epoch Number :  5
	 Training accuracy:  89.63460286458333
	 Validation accuracy  89.0911865234375
	 Epoch Loss  1.741786003112793
Epoch Number :  6
	 Training accuracy:  94.46268717447917
	 Validation accuracy  94.06880696614583
	 Epoch Loss  1.712183952331543
Epoch Number :  7
	 Training accuracy:  95.53093804

Epoch Number :  63
	 Training accuracy:  98.27548556857639
	 Validation accuracy  97.81534830729167
	 Epoch Loss  1.3445155620574951
Epoch Number :  64
	 Training accuracy:  98.17355685763889
	 Validation accuracy  97.78177897135417
	 Epoch Loss  1.3411431312561035
Epoch Number :  65
	 Training accuracy:  98.20631239149306
	 Validation accuracy  97.68208821614583
	 Epoch Loss  1.3399049043655396
Epoch Number :  66
	 Training accuracy:  97.95016818576389
	 Validation accuracy  97.50203450520833
	 Epoch Loss  1.3378103971481323
Epoch Number :  67
	 Training accuracy:  98.38175455729167
	 Validation accuracy  97.80924479166667
	 Epoch Loss  1.3380146026611328
Epoch Number :  68
	 Training accuracy:  98.32071940104167
	 Validation accuracy  97.77669270833333
	 Epoch Loss  1.3358142375946045
Epoch Number :  69
	 Training accuracy:  98.06491427951389
	 Validation accuracy  97.60945638020833
	 Epoch Loss  1.3327170610427856
Epoch Number :  70
	 Training accuracy:  98.37246365017361
	 Validati

Epoch Number :  125
	 Training accuracy:  98.60493977864583
	 Validation accuracy  97.5714111328125
	 Epoch Loss  1.2266778945922852
Epoch Number :  126
	 Training accuracy:  99.00187174479167
	 Validation accuracy  98.14066569010417
	 Epoch Loss  1.2222908735275269
Epoch Number :  127
	 Training accuracy:  99.04805501302083
	 Validation accuracy  97.99906412760417
	 Epoch Loss  1.2199712991714478
Epoch Number :  128
	 Training accuracy:  98.90557183159723
	 Validation accuracy  97.79296875
	 Epoch Loss  1.2170556783676147
Epoch Number :  129
	 Training accuracy:  98.62467447916667
	 Validation accuracy  97.89164225260417
	 Epoch Loss  1.2174785137176514
Epoch Number :  130
	 Training accuracy:  99.07307942708333
	 Validation accuracy  98.07169596354167
	 Epoch Loss  1.2098807096481323
Epoch Number :  131
	 Training accuracy:  98.9227294921875
	 Validation accuracy  97.9241943359375
	 Epoch Loss  1.207887053489685
Epoch Number :  132
	 Training accuracy:  98.78431532118056
	 Validation

Epoch Number :  187
	 Training accuracy:  99.28378634982639
	 Validation accuracy  98.14168294270833
	 Epoch Loss  1.14180588722229
Epoch Number :  188
	 Training accuracy:  99.19562445746527
	 Validation accuracy  97.93172200520833
	 Epoch Loss  1.1401599645614624
Epoch Number :  189
	 Training accuracy:  99.29884168836806
	 Validation accuracy  98.24564615885417
	 Epoch Loss  1.1362595558166504
Epoch Number :  190
	 Training accuracy:  99.36360677083333
	 Validation accuracy  98.0853271484375
	 Epoch Loss  1.1367409229278564
Epoch Number :  191
	 Training accuracy:  99.25564236111111
	 Validation accuracy  97.96630859375
	 Epoch Loss  1.135643482208252
Epoch Number :  192
	 Training accuracy:  99.16802300347223
	 Validation accuracy  98.20841471354167
	 Epoch Loss  1.135392427444458
Epoch Number :  193
	 Training accuracy:  99.15947808159723
	 Validation accuracy  98.14453125
	 Epoch Loss  1.1358880996704102
Epoch Number :  194
	 Training accuracy:  98.92232259114583
	 Validation acc

Epoch Number :  249
	 Training accuracy:  99.51117621527777
	 Validation accuracy  97.97342936197917
	 Epoch Loss  1.0855135917663574
Epoch Number :  250
	 Training accuracy:  99.30853949652777
	 Validation accuracy  97.94169108072917
	 Epoch Loss  1.0825648307800293
Epoch Number :  251
	 Training accuracy:  99.404296875
	 Validation accuracy  98.17728678385417
	 Epoch Loss  1.0858104228973389
Epoch Number :  252
	 Training accuracy:  99.4244384765625
	 Validation accuracy  97.94596354166667
	 Epoch Loss  1.0807527303695679
Epoch Number :  253
	 Training accuracy:  99.40504286024306
	 Validation accuracy  98.07637532552083
	 Epoch Loss  1.0830254554748535
Epoch Number :  254
	 Training accuracy:  99.371337890625
	 Validation accuracy  97.97810872395833
	 Epoch Loss  1.0806236267089844
Epoch Number :  255
	 Training accuracy:  99.40171983506944
	 Validation accuracy  97.9241943359375
	 Epoch Loss  1.0795643329620361
Epoch Number :  256
	 Training accuracy:  99.44288465711806
	 Validatio

In [ ]:
print(train_target.shape)

In [ ]:
print(train_target[44,:,:,:])

#### 

In [ ]:
print(output.shape)

In [ ]:
print(output[14,1,:,:,:])

In [ ]:
training_accuracy = np.array(100*(Ntrain_nb-train_errors)/Ntrain_nb)
validation_accurcy = np.array(100*(Nvalidation_nb-validation_errors)/Nvalidation_nb)

In [ ]:
plt.plot(training_accuracy)
plt.plot(validation_accurcy)
plt.show()

In [ ]:
test_visualisation = output[14,1,:,:,:].round()

In [ ]:
voxels = np.array(test_visualisation.data)

# and plot everything
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
ax.voxels(voxels)
fig.savefig('VoxelizedFinal.png')
fig.show()

In [ ]:
'''def summary(input_size, model):
        def register_hook(module):
            def hook(module, input, output):
                class_name = str(module.__class__).split('.')[-1].split("'")[0]
                module_idx = len(summary)

                m_key = '%s-%i' % (class_name, module_idx+1)
                summary[m_key] = OrderedDict()
                summary[m_key]['input_shape'] = list(input[0].size())
                summary[m_key]['input_shape'][0] = -1
                summary[m_key]['output_shape'] = list(output.size())
                summary[m_key]['output_shape'][0] = -1

                params = 0
                if hasattr(module, 'weight'):
                    params += th.prod(th.LongTensor(list(module.weight.size())))
                    if module.weight.requires_grad:
                        summary[m_key]['trainable'] = True
                    else:
                        summary[m_key]['trainable'] = False
                if hasattr(module, 'bias'):
                    params +=  th.prod(th.LongTensor(list(module.bias.size())))
                summary[m_key]['nb_params'] = params
                
            if not isinstance(module, nn.Sequential) and \
               not isinstance(module, nn.ModuleList) and \
               not (module == model):
                hooks.append(module.register_forward_hook(hook))
                
        dtype = th.cuda.FloatTensor
        
        # check if there are multiple inputs to the network
        if isinstance(input_size[0], (list, tuple)):
            x = [Variable(th.rand(1,*in_size)).type(dtype) for in_size in input_size]
        else:
            x = Variable(th.rand(1,*input_size)).type(dtype)
            
        print(x.shape)
        print(type(x[0]))
        # create properties
        summary = OrderedDict()
        hooks = []
        # register hook
        model.apply(register_hook)
        # make a forward pass
        model(x)
        # remove these hooks
        for h in hooks:
            h.remove()

        print('----------------------------------------------------------------')
        line_new = '{:>20}  {:>25} {:>15}'.format('Layer (type)', 'Output Shpae', 'Param #')
        print(line_new)
        print('================================================================')
        total_params = 0
        trainable_params = 0
        for layer in summary:
            ## input_shape, output_shape, trainable, nb_params
            line_new = '{:>20}  {:>25} {:>15}'.format(layer, summary[layer]['output_shape'], summary[layer]['nb_params'])
            total_params += summary[layer]['nb_params']
            if 'trainable' in summary[layer]:
                if summary[layer]['trainable'] == True:
                    trainable_params += summary[layer]['nb_params']
            print(line_new)
        print('================================================================')
        print('Total params: ' + str(total_params))
        print('Trainable params: ' + str(trainable_params))
        print('Non-trainable params: ' + str(total_params - trainable_params))
        print('----------------------------------------------------------------')
        return summary'''